In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np
import math
from scipy import signal
import soundfile as sf
import IPython.display as ipd

## Source Generation

In [2]:
noteMap = {
    'Ab': -11,
    'A': 12,
    'A#': 13,
    'Bb': 13,
    'B': 14,
    'C': 3,
    'C#': 4,
    'Db': 4,
    'D': 5,
    'D#': 6,
    'Eb': 6,
    'E': 7,
    'F': 8,
    'F#': 9,
    'Gb': 9,
    'G': 10,
    'G#': 11,
}
# Pass in tuple, for example ('A', 4)
def noteToHz(note):
    halfNotes = noteMap.get(note[0], None)
    if halfNotes is None:
        raise RuntimeError("Invalid note entered")
    midiNote = 21 + halfNotes + (12 * note[1])
    return math.pow(2, float(midiNote-69) / 12.0) * 440.0

In [3]:
sr = 44100

In [4]:
class ADSR(object):
    # Attack, Decay, Release
    def __init__(self, atk, dec, sus, rel):
        self.atk = int(atk * sr)
        self.dec = int(dec * sr)
        self.sus = sus
        self.rel = int(rel * sr)
        self.pos = 0
        self.lastVal = 0
        self.isNoteOn = True
        
    def noteOff(self):
        if self.isNoteOn:
            self.isNoteOn = False
            self.pos = 0
            
    def hasEnded(self):
        return not self.isNoteOn and self.pos > self.rel

    def render(self, buffer):
        for i in range(len(buffer)):
            val = 0
            if self.isNoteOn:
                if self.pos < self.atk:
                    val = float(self.pos) / float(self.atk)
                elif self.pos < (self.atk + self.dec):
                    a = float(self.pos - self.atk) / self.dec
                    val = (1-a) + self.sus * a
                else:
                    val = self.sus
                self.lastVal = val
            else:
                if self.pos < self.rel:
                    a = float(self.pos) / float(self.rel)
                    val = (1-a) * self.lastVal
            buffer[i] = buffer[i] * val
            self.pos += 1
                
class SineWave(object):
    def __init__(self, freq):
        self.freq = freq
        self.phi = 0
        
    def render(self, buffer):
        for i in range(len(buffer)):
            buffer[i] = math.sin(self.phi * 2 * math.pi)
            self.phi += float(self.freq) / float(sr)
            while self.phi > 1.0:
                self.phi -= 1.0
                
"""
    def makeSquareWaveAdditive(self, numHarmonics=10, buffer, amp=1, inSr):
        
        #replaced time with buffer
        #inSr is sample rate
        
        sqFreq = self.freq
        numSamples = int(time * inSr)
        sqLength = int(float(inSr) / float(sqFreq))
        amp = amp * 4 / math.pi

        phis = np.zeros(numHarmonics)
        phiDeltas = np.zeros(numHarmonics)
        for i in range(numHarmonics):
            harmonicFreq = float(sqFreq) * (float(i) * 2 + 1)
            phiDeltas[i] = harmonicFreq / float(inSr)

        tBuf = np.zeros(numSamples)
        buffer = np.zeros(numSamples)
        for i in range(numSamples):
            tBuf[i] = float(i) / float(inSr)
            for j in range(numHarmonics):
                buffer[i] += math.sin(2 * math.pi * phis[j]) * amp / (float(j) * 2 + 1)
                phis[j] += phiDeltas[j]
                while phis[j] >= 1:
                    phis[j] -= 1

        return (tBuf, buffer)
"""

In [5]:
class PlayingNote(object):
    def __init__(self, note, noteLength, amp):
        self.effects = []
        self.amp = amp
        self.lifetimeSamples = noteLength * sr
        self.envelope = ADSR(0.1, 0.2, 0.4, 0.1)
        self.source = SineWave(noteToHz(note))
        
    def hasEnded(self):
        return self.envelope.hasEnded()
        
    def render(self, buffer):
        outBuffer = np.zeros(len(buffer))
        self.source.render(outBuffer)
        outBuffer *= self.amp
        self.envelope.render(outBuffer)

        if self.lifetimeSamples > 0:
            self.lifetimeSamples -= len(buffer)
            if self.lifetimeSamples <= 0:
                self.envelope.noteOff()

        for effect in self.effects:
            effect.render(outBuffer)

        buffer += outBuffer

class Instrument(object):
    def __init__(self):
        self.playingNotes = []
        self.instrumentEffects = []
        
    def makePlayingNote(self, note):
        return PlayingNote(note, 2, 0.25)
    
    def playNote(self, note):
        self.playingNotes.append(self.makePlayingNote(note))
    
    def render(self, buffer):
        for playingNote in self.playingNotes:
            playingNote.render(buffer)
        for effect in self.instrumentEffects:
            effect.render(buffer)
            
        # Cleanup finished notes
        self.playingNotes = list(filter(lambda playingNote: not playingNote.hasEnded(),
                                  self.playingNotes))

class Orchestra(object):
    def __init__(self):
        self.instruments = [
            Instrument()
        ]
        self.globalEffects = []
#         self.globalEffects = [Delay()]
#         self.globalEffects = [SimpleChorus()]

    def render(self, buffer):
        for instrument in self.instruments:
            instrument.render(buffer)
        for effect in self.globalEffects:
            effect.render(buffer)

In [6]:
class Score(object):
    def __init__(self):
        self.orchestra = Orchestra()
        self.notes = [('A', 3), ('C', 4), ('E', 4), ('C', 4)]
        self.noteIdx = 0
        self.beatLengthInSamples = int(0.75 * sr)
        self.curBeatPos = 0
    
    def render(self, buffer):
        while self.curBeatPos <= 0:
            self.orchestra.instruments[0].playNote(
                self.notes[self.noteIdx]
            )
            self.noteIdx = (self.noteIdx + 1) % len(self.notes)
            self.curBeatPos += self.beatLengthInSamples
        self.orchestra.render(buffer)
        self.curBeatPos -= len(buffer)

## Modulated Effects

In [7]:
class RingBuffer(object):
    def __init__(self, maxDelay):
        self.maxDelay = maxDelay + 1
        self.buf = np.zeros(self.maxDelay)
        self.writeInd = 0

    def pushSample(self, s):
        self.buf[self.writeInd] = s
        self.writeInd = (self.writeInd + 1) % self.maxDelay

    def delayedSample(self, d):
        d = min(self.maxDelay - 1, max(0, d))
        i = ((self.writeInd + self.maxDelay) - d) % self.maxDelay
        return self.buf[i]

# Linear Interpolation
class LinearWrap(object):
    def __init__(self, it):
        self.it = it
        
    def __len__(self):
        return len(self.it)
        
    def __setitem__(self, inI, val):
        if type(inI) != int:
            raise RuntimeError('Can only write to integer values')
        self.it[inI] = val

    def __getitem__(self, inI):
        loI = math.floor(inI)
        hiI = math.ceil(inI)
        a = inI - loI
        inRange = lambda val: val >= 0 and val < len(self.it)
        loX = self.it[loI] if inRange(loI) else 0
        hiX = self.it[hiI] if inRange(hiI) else 0
        return loX * (1-a) + hiX * a

# Wrap inner iterable inside RingBuffer with float indexable array
class LinearRingBuffer(RingBuffer):
    def __init__(self, maxDelay):
        self.maxDelay = maxDelay + 1
        self.buf = LinearWrap(np.zeros(self.maxDelay))
        self.writeInd = 0

In [8]:
class Delay(object):
    def __init__(self):
        self.delayTime = 0.25
        self.delaySamps = int(self.delayTime * sr)
        self.ringBuf = RingBuffer(self.delaySamps)
        
    def render(self, buffer):
        for i in range(len(buffer)):
            s = buffer[i]
            self.ringBuf.pushSample(s)
            buffer[i] = s * 0.5 + self.ringBuf.delayedSample(self.delaySamps) * 0.5

In [9]:
class SimpleChorus(object) :
    def __init__(self):
        self.fmod = 1.5
        self.A = int(0.002 * sr)
        self.M = int(0.002 * sr)
        self.BL = 1.0
        self.FF = 0.7
        self.maxDelaySamps = self.M + self.A + 2
        self.ringBuf = LinearRingBuffer(self.maxDelaySamps)
        self.phi = 0
        
    def render(self, buffer):
        x = LinearWrap(buffer)
        deltaPhi = self.fmod/sr

        for i in range(len(buffer)):
            s = x[i]
            self.ringBuf.pushSample(s)
            delaySamps = int(math.sin(2 * math.pi * self.phi) * self.maxDelaySamps)
            buffer[i] = s * self.BL + self.ringBuf.delayedSample(delaySamps) * self.FF

            self.phi = self.phi + deltaPhi
            while self.phi >= 1:
                self.phi -= 1

## Audio Output

In [11]:
def playScore(filename, length, score):
    bufSize = 4096
    buffer = np.zeros(bufSize)
    lengthSamples = int(length * sr)
    numBlocks = int(lengthSamples / bufSize) + 1
    
    with sf.SoundFile(filename, 'wb', sr, 1) as f:
        for i in range(numBlocks):
            buffer *= 0
            score.render(buffer)
            f.write(buffer)

file_output = 'output/testScore.wav'
# file_output = 'output/testEcho.wav'
# file_output = 'output/testSimpleChorus.wav'
playScore(file_output, 10, Score())
ipd.Audio(file_output, rate=sr)